# Import các thư viện và khởi động Spark

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark.sql.functions as f 

# Thư viện FPGrowth để áp dụng giải thuật khai thác mẫu phổ biến và luật kết hợp
from pyspark.ml.fpm import FPGrowth

In [ ]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession


In [ ]:
spark = SparkSession.builder.getOrCreate()

# Đọc hai tập tin vào PySpark và sử dụng các phương thức tiền xử lý dữ liệu phù hợp để đưa dữ liệu về một DataFrame gồm hai cột theo đúng thứ tự là mã giao dịch (order_id) và danh sách sản phẩm (thể hiện bằng tên, không phải mã sản phẩm

## Đọc tập tin orders.csv và products.csv vào PySpark, lưu dưới dạng 2 dataframe orders và products

In [ ]:
orders = spark.read.load("../../data/Bai1/orders.csv", format="csv", header=True, delimiter=",")
products = spark.read.load("../../data/Bai1/products.csv", format="csv", header=True, delimiter=",")

## Dataframe orders gồm 4 cột order_id, product_id, add_to_cart_order, reordered


In [ ]:
orders.show()

## Dataframe products gồm 4 cột product_id, product_name, aisle_id, department_id


In [ ]:
products.show()

## Tiền xử lý dữ liệu

In [ ]:
# Dùng lệnh join để nhóm 2 dataframe dựa vào cột product_id của cả 2 dataframe
# Sau đó nhóm theo cột order_id bằng lệnh groupby
# sau đó tạo cột products chứa danh sách các product_name thuộc dòng có order_id tương ứng bằng hàm collect_list của pyspark.sql
# Đưa cột order_id từ chuỗi về loại số thực sau đó sắp xếp cột này từ bé đến lớn (lệnh orderBy) để dễ theo dõi kết quả

df = orders.join(products,
               products.product_id == orders.product_id,
               "inner")\
.groupby('order_id').agg(f.collect_list("product_name").alias('products'))\
.withColumn("order_id", orders["order_id"] - 0)\
.orderBy("order_id")
df.show()
print(f'Dataframe kết quả có tổng cộng {df.count()} dòng')

# Áp dụng giải thuật khai thác mẫu phổ biến và luật kết hợp trong gói pyspark.ml.fpm. Thử nghiệm với một số bộ giá trị ngưỡng support và confidence.


## (minSupport,minConfidence) = (0.2,0.2)

In [ ]:
fpGrowth = FPGrowth(itemsCol="products", minSupport=0.2, minConfidence=0.2)

model = fpGrowth.fit(df)

# Tập mẫu phổ biến patternsDF
patternsDF = model.freqItemsets

# Tập luật kết hợp associationRules
rulesDF = model.associationRules

print(f'Có {patternsDF.count()} mẫu phổ biến và {rulesDF.count()} luật kết hợp đối với bộ giá trị này')

patternsDF.show(patternsDF.count(), False)

rulesDF.show(rulesDF.count(), False)

## (minSupport,minConfidence) = (0.1,0.1)

In [ ]:
fpGrowth = FPGrowth(itemsCol="products", minSupport=0.1, minConfidence=0.1)

model = fpGrowth.fit(df)
# Tập mẫu phổ biến patternsDF
patternsDF = model.freqItemsets
# Tập luật kết hợp associationRules
rulesDF = model.associationRules

print(f'Có {patternsDF.count()} mẫu phổ biến và {rulesDF.count()} luật kết hợp đối với bộ giá trị này')

patternsDF.show(patternsDF.count(), False)

rulesDF.show(rulesDF.count(), False)

## (minSupport,minConfidence) = (0.01,0.005)

In [ ]:
fpGrowth = FPGrowth(itemsCol="products", minSupport=0.01, minConfidence=0.005)
model = fpGrowth.fit(df)
# get the frequent itemsets
patternsDF = model.freqItemsets
# get the association rules
rulesDF = model.associationRules

print(f'Có {patternsDF.count()} mẫu phổ biến và {rulesDF.count()} luật kết hợp đối với bộ giá trị này')

patternsDF.show(patternsDF.count(), False)

rulesDF.show(rulesDF.count(), False)

## (minSupport,minConfidence) = (0.005,0.001)

In [ ]:
fpGrowth = FPGrowth(itemsCol="products", minSupport=0.005, minConfidence=0.001)
model = fpGrowth.fit(df)
# get the frequent itemsets
patternsDF = model.freqItemsets
# get the association rules
rulesDF = model.associationRules

print(f'Có {patternsDF.count()} mẫu phổ biến và {rulesDF.count()} luật kết hợp đối với bộ giá trị này')

patternsDF.show(patternsDF.count(), False)

rulesDF.show(rulesDF.count(), False)

# Bạn có nhận thấy vấn đề gì về hình thức của các luật được tìm thấy hay không (ta không cần quan tâm ngữ nghĩa của dữ liệu). Nếu có, hãy khắc phục điều này

## Vấn đề 1:
- Với minsupport và minconfidence lớn, cụ thể là từ 0.2 trở lên thì không tìm được mẫu phổ biến cũng như luật nào 
- Để khắc phục thì ta có thể giảm cả 2 tham số xuống dưới ít nhất 0.1

## Vấn đề 2:
- Khi đã giảm cả 2 tham số thì ta có thể thấy ở phần antecedent và consequent đều chỉ có 1 sản phẩm ở tất cả các luật
- Để khắc phục điều này, ta có thể giảm minSupport xuống 0.005 và giảm minConfindence xuống ít nhất là 0.001 để ở phần antecedent xuất hiện các luật có từ 2 sản phẩm trở lên như Organic Strawberries, Bag of Organic Bananas -> Organic Hass Avocado hay Organic Hass Avocado, Bag of Organic Bananas -> Organic Strawberries